In [1]:
from contgan_images.contgan import ContGAN
from contgan_images.utils import load_weights

device = "cuda:7"
model = ContGAN(image_shape=(3, 32, 32), decoder_type="fixed")
load_weights(model, "model.pt")

model.to(device)
model.eval();

/home/mavrinti/contgan/src/utils.py:29: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  loaded_state_dict = th.load(weights_path, map_location="cpu")


In [2]:
from contgan_images.utils import jvp

import torch.nn as nn
import torch as th

from functools import partial

class ODEfunc(nn.Module):
    def __init__(self, model):
        super().__init__()

        self.model = model

    def forward(self, t, x):
        t = t.unsqueeze(0)
        with th.no_grad():
            x_hat, eps_hat = self.model.decoder(x_t=x, t=t)

        t_ = t.detach().clone().requires_grad_()
        encoder_ = partial(self.model.encoder, x_0=x_hat, eps=eps_hat)
        _, x_t_hat_grad = jvp(encoder_, t_)

        return x_t_hat_grad
    
ode_func = ODEfunc(model)

In [3]:
from torchdiffeq import odeint_adjoint as odeint
import numpy as np

out = odeint(ode_func, th.randn((64, 3, 32, 32), device=device), th.tensor(np.linspace(1., 0., 2), device=device))
out = (out[1].clip(-1., 1.) + 1.) / 2.

In [ ]:
from torchvision.utils import save_image

save_image(out, "out/ode_sample.png")

---